<a href="https://colab.research.google.com/github/NatLibFi/Annif/blob/kvp2019-workshop/arviointipohjien_luonti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install gspread-pandas gspread-formatting

In [0]:
! mkdir -p  /root/.config/gspread_pandas/
! echo '{"installed":{"client_id":"257572009167-uai362klp88kr3hluabtoujf2hnd7fr6.apps.googleusercontent.com","project_id":"formal-archive-254810","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"x554t-fVXmFDrENKrXu8GuKy","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}' > /root/.config/gspread_pandas/google_secret.json

In [0]:
import pandas as pd
import random
from gspread_pandas import Spread, Client
from gspread_formatting import cellFormat, format_cell_range, set_frozen, color, textFormat

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 100) 

In [111]:
# Fetch kuvailut

client = Client()
spreads = client.find_spreadsheet_files_in_folders('Dokumenttien kuvailut')

df_kuvailut1 = pd.DataFrame()
df_kuvailut2 = pd.DataFrame()
df_kuvailut3 = pd.DataFrame()

for kuvailu_id, spread in enumerate(spreads['Dokumenttien kuvailut']):
  print(spread['name'])
  # Tietokirjat:
  if spread['name'].startswith("Kuvailut Annif"):
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut1 = df_kuvailut1.append(df_sheet)
  elif spread['name'].startswith("kuvailut_tieto"):
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.rename(columns={'isbn': 'url kokotekstiin'}, inplace=True)
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut2 = df_kuvailut2.append(df_sheet)
  else:
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut3 = df_kuvailut3.append(df_sheet)    
  

Copy of Kuvailut Fennica
Copy of Kuvailut K27
Kuvailut Annif-FastText
Kuvailut Annif-PAV
Kuvailut Annif-TFIDF
Kuvailut Annif-Maui
jyx_kuvailut Juho
SKT1 KL Copy of Kuvailut


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


jyx_kuvailut Mona
jyx_kuvailut Osma
jyx_kuvailut_orig
kuvailut_tieto Osma
Copy of Kuvailut K07
Copy of Kuvailut K16
Copy of Kuvailut K31
kuvailut_tieto Mona
kuvailut_tieto Fennica
kuvailut_tieto Juho
SKT4 VL Copy of Kuvailut 
SKT3 JM Kuvailut Copy of 
SKT2 PM Copy of Kuvailut 
Copy of Kuvailut K02
Copy of Kuvailut K17
Copy of Kuvailut K05
Copy of Kuvailut K14
Copy of Kuvailut K11
Copy of Kuvailut K08
Copy of Kuvailut K32
Copy of Kuvailut K23
Copy of Kuvailut K29
Copy of Kuvailut K20
Copy of Kuvailut K26
Copy of Kuvailut K19
Copy of Kuvailut K10
Copy of Kuvailut K38
Copy of Kuvailut K13
Copy of Kuvailut K35
Copy of Kuvailut K40
Copy of Kuvailut K28
Copy of Kuvailut K25
Copy of Kuvailut K37
Copy of Kuvailut K22
Copy of Kuvailut K18
Copy of Kuvailut K06
Copy of Kuvailut K12
Copy of Kuvailut K03
Copy of Kuvailut K09
Copy of Kuvailut K33
Copy of Kuvailut K39
Copy of Kuvailut K21
Copy of Kuvailut K36
Copy of Kuvailut K04
Copy of Kuvailut K01
Copy of Kuvailut K34
Copy of Kuvailut K24
Copy of 

In [113]:
# Preprocess 1

df_kuvailut = pd.concat([df_kuvailut2, df_kuvailut3])
cols_to_keep = ['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi']
df_kuvailut = df_kuvailut[cols_to_keep]
df_kuvailut.rename(columns={'id': 'did'}, inplace=True)
df_kuvailut[['did', 'nimeke', 'url kokotekstiin']] = df_kuvailut[['did', 'nimeke', 'url kokotekstiin']].mask(df_kuvailut[['did', 'nimeke', 'url kokotekstiin']]=='', None).ffill()

# Append Annif indexed
df_kuvailut1.rename(columns={'id': 'did'}, inplace=True)
df_kuvailut1['did'] = df_kuvailut1['did'].mask(df_kuvailut1['did']=='', None).ffill()
df_kuvailut = df_kuvailut.append(
    df_kuvailut1.merge(
        df_kuvailut[['did', 'nimeke', 'url kokotekstiin']].drop_duplicates(),
        on='did'
    ))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
# Set types of index source method
#ihminen = ['kuvailut_tieto Mona', 'jyx_kuvailut Mona', 'jyx_kuvailut Juho', 'Copy of Kuvailut K']
annif_avusteiset = ['kuvailut_tieto Osma', 'kuvailut_tieto Juho', 'jyx_kuvailut_orig']
annif = 'Kuvailut Annif-'
fennica = ['kuvailut_tieto Fennica', 'Copy of Kuvailut Fennica']

df_kuvailut['tyyppi'] = df_kuvailut['tiedostonimi'].apply(lambda x:
                                                          'annif_avusteinen' if x in annif_avusteiset
                                                          else (
                                                              'fennica' if x in fennica
                                                              else (
                                                                  'annif' if 'Kuvailut Annif-' in x 
                                                                  else 'ihminen'
                                                            )))

# Remove example:
df_kuvailut = df_kuvailut[df_kuvailut.did != 'D00']

# Correct url for document D20:
df_kuvailut.loc[df_kuvailut.did == 'D20', 'url kokotekstiin'] = 'https://www.doria.fi/bitstream/handle/10024/130803/yyti_stv_201600_2016_net_p2.pdf?sequence=2&isAllowed=y'
#df_kuvailut1[df_kuvailut1.did == 'D01']

In [118]:
# Create header and footer to insert around kuvailu 

df_ylaosa = pd.DataFrame({'did': [''], 'kid': [''], 'aiheet': ['aiheet'], 'poistettavat': ['poista = x'], 'empty1': [''], 'empty2': ['']})

df_alaosa0 = pd.DataFrame({'did': [''], 'aiheet': [''], 'kid': ['']})
df_alaosa1 = pd.DataFrame({'did': [''], 'empty2': ['arvosana'], 'kid': ['']})
df_alaosa2 = pd.DataFrame({'did': ['']*4, 'empty1': ['Kuvailu on kattava', 'Käytetyt asiasanat ovat sisällön kannalta merkityksellisiä', 'Kuvailu on laadukas', 'Vapaa kenttä'], 'kid': ['']*4})
df_alaosa = pd.concat([df_alaosa0, df_alaosa1, df_alaosa2, df_alaosa0, df_alaosa0])


display(df_ylaosa)
display(df_alaosa)

# Formatting
fmt = cellFormat(
    horizontalAlignment='RIGHT'
    )
fmt_header = cellFormat(
    backgroundColor=color(144/256., 238/256., 144/256.),
    textFormat=textFormat(bold=True) #, foregroundColor=color(1, 0, 1)),
    )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,did,kid,aiheet,poistettavat,empty1,empty2
0,,,aiheet,poista = x,,


,aiheet,did,empty1,empty2,kid
0,,,NaN,NaN,
0,NaN,,NaN,arvosana,
0,NaN,,Kuvailu on kattava,NaN,
1,NaN,,Käytetyt asiasanat ovat sisällön kannalta merkityksellisiä,NaN,
2,NaN,,Kuvailu on laadukas,NaN,
3,NaN,,Vapaa kenttä,NaN,
0,,,NaN,NaN,
0,,,NaN,NaN,


In [122]:
# Create files and sheets

min_num_aiheet = 1
max_num_ihmiskuvailut = 3
num_arvioija = 5
num_sheets = 10  # One sheet for one document

include_types = ['fennica', 'annif', 'annif_avusteinen']
columns = ['did', 'kid', 'aiheet'] #, 'tiedostonimi']

dokumentit = [(did, df_dokumentti) for did, df_dokumentti in df_kuvailut.groupby('did')]

for arvioija_ind in range(num_arvioija):
    print('arvioija_ind: ', arvioija_ind)
    spread = Spread('arvioija_' + str(arvioija_ind) , create_spread=True)
    spread.move('Kivepa2019 Aaveita koneessa (kivepa-aaveita)/Arvioijakohtaiset taulukot')
    #random.shuffle(dokumentit)

    cnt_sheet = 0
    for did, df_dokumentti in dokumentit:
        spread.open_sheet(did, create=True)
        df_pohja = pd.DataFrame(columns=columns)
        print('document_id: ', did)
        kuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.tyyppi.iloc[0] in include_types
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        ihmiskuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.tyyppi.iloc[0] == 'ihminen'
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        random.shuffle(ihmiskuvailut)
        ihmiskuvailut = ihmiskuvailut[:max_num_ihmiskuvailut]
        kuvailut.extend(ihmiskuvailut)
        random.shuffle(kuvailut)
        
        df_header = df_dokumentti.iloc[0][['did', 'nimeke', 'url kokotekstiin']].to_frame()
        spread.df_to_sheet(df_header, headers=False, index=False)

        for df_dokumentti_kuvailu in kuvailut:
            df_pohja = df_pohja.append(df_ylaosa.append(df_dokumentti_kuvailu[columns], sort=False).append(df_alaosa, sort=False).fillna(''), sort=False)

        spread.df_to_sheet(df_pohja, index=False, headers=False, start=(4, 1))
        format_cell_range(spread.sheet, 'D1:E' + str(len(df_pohja)), fmt)
        set_frozen(spread.sheet, rows=3)
        format_cell_range(spread.sheet, 'A1:F3', fmt_header)
        
        cnt_sheet += 1
        if cnt_sheet >= num_sheets:
            break
    spread.delete_sheet('Sheet1')
    dokumentit = dokumentit[1:] + [dokumentit[0]]  # Rotate order from front to back


arvioija_ind:  0
document_id:  D01
document_id:  D02
document_id:  D03
document_id:  D04
document_id:  D05
document_id:  D06
document_id:  D07
document_id:  D08
document_id:  D09
document_id:  D10
arvioija_ind:  1
document_id:  D02
document_id:  D03
document_id:  D04
document_id:  D05
document_id:  D06
document_id:  D07
document_id:  D08
document_id:  D09
document_id:  D10
document_id:  D11
arvioija_ind:  2
document_id:  D03
document_id:  D04
document_id:  D05
document_id:  D06
document_id:  D07
document_id:  D08
document_id:  D09
document_id:  D10
document_id:  D11
document_id:  D12
arvioija_ind:  3
document_id:  D04
document_id:  D05
document_id:  D06
document_id:  D07
document_id:  D08
document_id:  D09
document_id:  D10
document_id:  D11
document_id:  D12
document_id:  D13
arvioija_ind:  4
document_id:  D05
document_id:  D06
document_id:  D07
document_id:  D08
document_id:  D09
document_id:  D10
document_id:  D11
document_id:  D12
document_id:  D13
document_id:  D14


In [132]:
# Save kuvailut data for use in analysis

from google.colab import drive
drive.mount('drive')

# Drop rows without aihe (corresponds to min_num_aiheet = 1)
df_kuvailut[df_kuvailut.aiheet != ''].to_csv('kuvailut.csv')
!cp kuvailut.csv drive/My\ Drive/Kivepa2019\ Aaveita\ koneessa\ \(kivepa-aaveita\)/


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
